In [3]:
import requests as req
import pandas as pd
import copy

In [4]:
req_url = 'https://einfo.ceproas.cz/cepro_portal_ws/rest/common/prox/mobileData'
req_header = {'authorization': 'Basic bW9iYXA6RVdpa0Ey',
              'content-type': 'application/json; charset=UTF-8',
              'accept-encoding': 'gzip',
              'user-agent': 'okhttp/4.9.0'}
json_body_encyclopedia = '{"encyklopedie": {"full": true}, "version": 5}'
json_body_stations = '{"search": {"exclude_cs_ceny": false,"exclude_cs_kvalita": false},"version": 5}'

In [4]:
try:
    response_gas_stations = req.post(url=req_url, headers=req_header, data=json_body_stations, timeout = 10)
    response_encyclopedia = req.post(url=req_url, headers=req_header, data=json_body_encyclopedia, timeout = 10)
    response_gas_stations.raise_for_status()
    response_encyclopedia.raise_for_status()
except req.exceptions.Timeout as errrt:
    print("ERROR : ceproas.cz API - Request Time out. CEPRO server is down.")
except req.exceptions.RequestException:
    print('ERROR : Something went wrong.')

In [5]:
# pd.to_pickle(response_gas_stations, '_data/response_gas_stations.pickle')
# pd.to_pickle(response_encyclopedia, '_data/response_encyclopedia.pickle')

response_gas_stations = pd.read_pickle('_data/response_gas_stations.pickle')
response_encyclopedia = pd.read_pickle('_data/response_encyclopedia.pickle')

In [33]:
# for development
all_gs = copy.deepcopy(response_gas_stations.json())
encyclopedia_info = copy.deepcopy(response_encyclopedia.json())

# check for errors in json files
if all_gs['success'] is not True:
    error_msg_all_gs = all_gs['Error']['errorText']
    print(error_msg_all_gs)

if encyclopedia_info['success'] is not True:
    error_msg_encyclopedia_info = encyclopedia_info['Error']['errorText']
    print(error_msg_encyclopedia_info)

# gas stations
data_gs = all_gs['Data']

# fuel prices
gs_prices = pd.concat([
    pd.DataFrame(data_gs['cs_ceny']).explode('ceny').reset_index()
    , pd.DataFrame(pd.DataFrame(data_gs['cs_ceny']).explode('ceny')['ceny'].tolist()).reset_index()
    ]
    , axis=1)[['kod_cs', 'cena', 'kod_produkt']]

# product list
products_list = pd.DataFrame(data_gs['cis_prod_list'])

# product quality
gs_product_quality = pd.DataFrame()
for gs_product in data_gs['cs_kvalita']:
    gs_number = gs_product['kod_cs']
    product_quality_gs = pd.DataFrame(pd.DataFrame(gs_product['produkty']).explode('parametry')['parametry'].to_list())
    product_quality_gs['kod_cs'] = gs_number
    gs_product_quality = pd.concat([gs_product_quality,product_quality_gs], axis='index')

# gas station location
gs_adress_wo_gps = pd.DataFrame(data_gs['cs_fix_list']).reset_index()
gs_adress = pd.concat([gs_adress_wo_gps, pd.DataFrame(gs_adress_wo_gps['GPS'].tolist())], axis=1)
del(gs_adress_wo_gps)

# Pruct quality list
product_quality_list = data_gs['cis_kvalita_list']


# encyclopedia
# info = encyclopedia_info['Encyklopedie']
# for i in info['tree']:
#     if i['parent'] == 'root':
#         root = pd.DataFrame(i['children'])
#     if i['parent'] == 'produkt':
#         product = pd.DataFrame(i['children'])
#     if i['parent'] == 'kvalita':
#         quality = pd.DataFrame(i['children'])
#     if i['parent'] == 'kvalita:natural95':
#         quality_95 = pd.DataFrame(i['children'])
#     if i['parent'] == 'kvalita:optimal_diesel':
#         quality_diesel = pd.DataFrame(i['children'])

In [46]:
# prices_gs # cena paliva
# product_quality # hodnoty kvality pro kazdou benzinu
# product_quality_list  # list parmaetru kvality
# products # produkt ciselnik
gs_adress # adresa,kraje GPS


,index,psc,obec,kraj,cislo,nazev_dlouhy,ulice,okres,GPS,nazev_kratky,kod_cs,long_dec,lat_dec,long,lat
0,0,36301,Hájek,Karlovarský kraj,,Hájek (Karlovy V.),"areál skladu Čepro, a.s.",Karlovy Vary,"{'long_dec': 12.9286722222, 'lat_dec': 50.2784...",Hájek (Karlovy V.),21,12.928672,50.278414,"12°55'43.22"" E","50°16'42.29"" N"
1,1,38743,Bělčice,Jihočeský kraj,,Bělčice,"areál skladu Čepro, a.s.",Strakonice,"{'long_dec': 13.8956472222, 'lat_dec': 49.5122...",Bělčice,22,13.895647,49.512267,"13°53'44.33"" E","49°30'44.16"" N"
2,2,37382,Včelná,Jihočeský kraj,,Včelná,"areál skladu Čepro, a.s.",České Budějovice,"{'long_dec': 14.4432611111, 'lat_dec': 48.9205...",Včelná,31,14.443261,48.920586,"14°26'35.74"" E","48°55'14.11"" N"
3,3,41108,Štětí,Ústecký kraj,,Hněvice,"areál skladu Čepro, a.s.",Litoměřice,"{'long_dec': 14.3503305556, 'lat_dec': 50.4419...",Hněvice,41,14.350331,50.441964,"14°21'1.19"" E","50°26'31.07"" N"
4,4,25091,Zeleneč,Středočeský kraj,,Mstětice,"areál skladu Čepro, a.s.",Praha - východ,"{'long_dec': 14.6938027778, 'lat_dec': 50.1424...",Mstětice,51,14.693803,50.142486,"14°41'37.69"" E","50°8'32.95"" N"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,203,27034,Čistá,Středočeský kraj,,Čistá u Rakovníka,,Rakovník,"{'long_dec': 13.5798777778, 'lat_dec': 50.0288...",Čistá u Rakovníka,620,13.579878,50.028825,"13°34'47.56"" E","50°1'43.77"" N"
204,204,35733,Loket,Karlovarský kraj,,Loket n. Ohří,TGM Masaryka,Sokolov,"{'long_dec': 12.7513111111, 'lat_dec': 50.1886...",Loket n. Ohří,621,12.751311,50.188692,"12°45'4.72"" E","50°11'19.29"" N"
205,205,35703,Svatava,Karlovarský kraj,,Sokolov,Kraslická,Sokolov,"{'long_dec': 12.6351722222, 'lat_dec': 50.1889...",Sokolov,622,12.635172,50.188947,"12°38'6.62"" E","50°11'20.21"" N"
206,206,34815,Planá,Plzeňský kraj,,Planá u Mar.Lázní,Plzeňská,Tachov,"{'long_dec': 12.7366638889, 'lat_dec': 49.8627...",Planá u Mar.Lázní,624,12.736664,49.862764,"12°44'11.99"" E","49°51'45.95"" N"
